# Pré-traitement du fichier consommation de 2011 à 2019

## Import des librairies

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

from utils import normalize


## Chargement des données

In [3]:
path = 'conso_full.csv'

df = pd.read_csv(path, sep=';')

C:\Users\A1BY3ZZ\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8,15,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.columns = [normalize(col) for col in df.columns]
df = df.rename(columns={'consommation_(mwh)': 'consommation'})
df.head()

,operateur,annee,filiere,code_categorie_consommation,libelle_categorie_consommation,code_grand_secteur,libelle_grand_secteur,code_naf,libelle_secteur_naf,consommation,nombre_de_points,indice_qualite,thermosensibilite_(mwh/degre-jour),part_thermosensible,nombre_de_mailles_secretisees,code_commune,libelle_commune,code_epci,libelle_epci,code_departement,libelle_departement,code_region,libelle_region
0,GRDF,2012,Gaz,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,803.0,54,0,0.0,0.0,0,47043,Buzet-sur-Baïse,200068948,CC Albret Communauté,47,Lot-et-Garonne,75,Nouvelle-Aquitaine
1,GRDF,2012,Gaz,ENT,Entreprises,T,Tertiaire,NaN,0,29935.0,5,1,0.0,0.0,0,47049,Casseneuil,200023307,CA du Grand Villeneuvois,47,Lot-et-Garonne,75,Nouvelle-Aquitaine
2,GRDF,2012,Gaz,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,5075.0,231,0,0.0,0.0,0,47051,Castelculier,200035459,CA d'Agen,47,Lot-et-Garonne,75,Nouvelle-Aquitaine
3,GRDF,2012,Gaz,ENT,Entreprises,I,Industrie,NaN,0,15882.0,1,1,0.0,0.0,0,47054,Castelmoron-sur-Lot,244701405,CC Lot et Tolzac,47,Lot-et-Garonne,75,Nouvelle-Aquitaine
4,GRDF,2012,Gaz,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,4264.0,137,0,0.0,0.0,0,47054,Castelmoron-sur-Lot,244701405,CC Lot et Tolzac,47,Lot-et-Garonne,75,Nouvelle-Aquitaine


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492458 entries, 0 to 1492457
Data columns (total 23 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   operateur                           1492458 non-null  object 
 1   annee                               1492458 non-null  int64  
 2   filiere                             1492458 non-null  object 
 3   code_categorie_consommation         1492458 non-null  object 
 4   libelle_categorie_consommation      1449250 non-null  object 
 5   code_grand_secteur                  1492458 non-null  object 
 6   libelle_grand_secteur               1492458 non-null  object 
 7   code_naf                            369611 non-null   float64
 8   libelle_secteur_naf                 1492458 non-null  object 
 9   consommation                        1492458 non-null  float64
 10  nombre_de_points                    1492458 non-null  int64  
 11  indice_qual

In [6]:
df.isna().mean()

operateur                             0.000000
annee                                 0.000000
filiere                               0.000000
code_categorie_consommation           0.000000
libelle_categorie_consommation        0.028951
code_grand_secteur                    0.000000
libelle_grand_secteur                 0.000000
code_naf                              0.752347
libelle_secteur_naf                   0.000000
consommation                          0.000000
nombre_de_points                      0.000000
indice_qualite                        0.000000
thermosensibilite_(mwh/degre-jour)    0.000000
part_thermosensible                   0.000000
nombre_de_mailles_secretisees         0.000000
code_commune                          0.000000
libelle_commune                       0.000036
code_epci                             0.000036
libelle_epci                          0.000036
code_departement                      0.000000
libelle_departement                   0.000000
code_region  

## Nettoyage des colonnes

On supprime les colonnes qui ne nous intéressent pas ici

In [7]:
df.columns

Index(['operateur', 'annee', 'filiere', 'code_categorie_consommation',
       'libelle_categorie_consommation', 'code_grand_secteur',
       'libelle_grand_secteur', 'code_naf', 'libelle_secteur_naf',
       'consommation', 'nombre_de_points', 'indice_qualite',
       'thermosensibilite_(mwh/degre-jour)', 'part_thermosensible',
       'nombre_de_mailles_secretisees', 'code_commune', 'libelle_commune',
       'code_epci', 'libelle_epci', 'code_departement', 'libelle_departement',
       'code_region', 'libelle_region'],
      dtype='object')

In [8]:
print(df['code_categorie_consommation'].unique())
print(df['libelle_categorie_consommation'].unique())

# on peut garder les libellés -> on remplacera NaN par 'Non renseigné'

['RES-PRO' 'ENT' 'X' 'PRO' 'RES' 'ENT_PRO']
['Résidentiel + Petits Professionnels' 'Entreprises' nan
 'Petits professionels' 'Résidentiel']


In [9]:
print(df['code_naf'].unique())
print(df['libelle_secteur_naf'].unique())

# intéressant mais sans doute un peu précis dans un premier temps

[nan 84. 25. 61. 55. 77.  1. 10. 87. 46. 35. 70. 94. 45. 68. 16. 28. 38.
 56. 31. 33. 36. 29. 47. 43. 86. 96. 18. 41. 37. 49. 52. 65. 82. 85. 90.
 98. 20. 69. 71. 95. 42. 66. 99. 23. 93. 63. 74. 80. 21. 22.  9. 19. 81.
 17. 59. 64. 91.  8. 53. 26. 11. 60. 88. 62. 14. 32. 72. 73. 78. 79. 30.
 15. 58. 75. 39. 27. 50. 51. 92. 13.  2. 24.  3.  7. 97.  6. 12.  5.]
[0 '0' 'Administration publique et défense ; sécurité sociale obligatoire'
 "Fabrication de produits métalliques, à l'exception des machines et des équipements"
 'Télécommunications' 'Hébergement'
 'Activités de location et location-bail'
 'Culture et production animale, chasse et services annexes'
 'Industries alimentaires' 'Hébergement médico-social et social'
 "Commerce de gros, à l'exception des automobiles et des motocycles"
 "Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné"
 'Activités des sièges sociaux ; conseil de gestion'
 'Activités des organisations associatives'
 "Commerce et réparatio

In [10]:
cols_to_drop = ['operateur', 'code_categorie_consommation',
       'code_grand_secteur', 'code_naf', 'libelle_secteur_naf',
        'indice_qualite',
       'thermosensibilite_(mwh/degre-jour)', 'part_thermosensible',
       'nombre_de_mailles_secretisees', 
       'code_epci', 'libelle_epci', 'libelle_departement',
       'code_region', 'libelle_region']

# on garde libellé et code pour les communes pour plus facilement comprendre si il y a des soucis

df = df[[col for col in df.columns if col not in cols_to_drop]]

df.head()

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
0,2012,Gaz,Résidentiel + Petits Professionnels,Résidentiel,803.0,54,47043,Buzet-sur-Baïse,47
1,2012,Gaz,Entreprises,Tertiaire,29935.0,5,47049,Casseneuil,47
2,2012,Gaz,Résidentiel + Petits Professionnels,Résidentiel,5075.0,231,47051,Castelculier,47
3,2012,Gaz,Entreprises,Industrie,15882.0,1,47054,Castelmoron-sur-Lot,47
4,2012,Gaz,Résidentiel + Petits Professionnels,Résidentiel,4264.0,137,47054,Castelmoron-sur-Lot,47


In [11]:
df.columns

Index(['annee', 'filiere', 'libelle_categorie_consommation',
       'libelle_grand_secteur', 'consommation', 'nombre_de_points',
       'code_commune', 'libelle_commune', 'code_departement'],
      dtype='object')

## Exclusion de la Corse et des DOM-TOM

In [12]:
df = df[df['code_departement'].astype(str).isin(['2A', '2B']) == False]

In [13]:
df.loc[:, 'code_departement'] = df.loc[:, 'code_departement'].astype(int)

In [14]:
df = df[df['code_departement'] <= 95]

In [15]:
df['code_commune'] = df['code_commune'].astype(str)
df['code_commune'] = df['code_commune'].str.replace(' ', '').astype(int)

In [16]:
df[df['code_commune'].isna()]

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement


## Traitement des villes à Arrondissement

In [18]:
number = re.compile('^(Paris )?(Marseille )?(Lyon )?')
number = re.compile('[0-9]+')

print(number.findall('Paris 1er arrondissement'))
print(number.findall('Marseille 1er arrondissement'))
print(number.findall('Lyon 1er arrondissement'))
print(number.findall('Rennes'))


['1']
['1']
['1']
[]


In [19]:
df_arrondissements = df[df['libelle_commune'].astype(str).apply(lambda x: len(number.findall(x))> 0)]

In [20]:
df_arrondissements['libelle_commune'].unique()

array(['Lyon 2e Arrondissement', 'Lyon 3e Arrondissement',
       'Lyon 4e Arrondissement', 'Lyon 5e Arrondissement',
       'Lyon 6e Arrondissement', 'Lyon 7e Arrondissement',
       'Lyon 8e Arrondissement', 'Lyon 9e Arrondissement',
       'Paris 1er Arrondissement', 'Paris 3e Arrondissement',
       'Paris 4e Arrondissement', 'Paris 5e Arrondissement',
       'Paris 6e Arrondissement', 'Paris 8e Arrondissement',
       'Paris 9e Arrondissement', 'Paris 10e Arrondissement',
       'Paris 11e Arrondissement', 'Paris 12e Arrondissement',
       'Paris 14e Arrondissement', 'Paris 15e Arrondissement',
       'Paris 16e Arrondissement', 'Paris 17e Arrondissement',
       'Paris 18e Arrondissement', 'Paris 20e Arrondissement',
       'Marseille 11e Arrondissement', 'Marseille 15e Arrondissement',
       'Marseille 1er Arrondissement', 'Marseille 2e Arrondissement',
       'Marseille 4e Arrondissement', 'Marseille 5e Arrondissement',
       'Marseille 6e Arrondissement', 'Marseille 7e Arro

In [21]:
num_lines_arrondissements = df_arrondissements.shape[0]
num_lines_arrondissements

9339

In [22]:
# recréation du code commune pour ces villes: on prend le code postal
df_arrondissements.loc[:, 'code_commune'] = df_arrondissements.loc[:, 'code_departement'] * 1000

C:\Users\A1BY3ZZ\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [23]:
df_arrondissements['code_commune'].unique()

array([69000, 75000, 13000])

In [24]:
# on vérifie que ces codes n'existent pas déja dans le dataset de départ
df[df['code_commune'].isin(df_arrondissements['code_commune'].unique())]

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement


In [25]:
# remplacement des noms des villes
df_arrondissements.loc[:, 'libelle_commune'] = df_arrondissements.loc[:, 'libelle_commune'].apply(lambda x: number.split(x)[0].replace(' ', ''))

C:\Users\A1BY3ZZ\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [26]:
df_arrondissements['libelle_commune'].unique()

array(['Lyon', 'Paris', 'Marseille'], dtype=object)

In [27]:
df_arrondissement = df_arrondissements.groupby(
    [col for col in df.columns if col not in ['consommation', 'nombre_de_points']]
).sum().reset_index()

In [28]:
# suppression dans le dataset de départ

initial_shape = df.shape[0]

df = df[df['libelle_commune'].astype(str).apply(lambda x: len(number.findall(x))== 0)]

posterior_shape = df.shape[0]

print(initial_shape)
print(posterior_shape)
print(initial_shape - num_lines_arrondissements)

1480926
1471587
1471587


In [29]:
# concaténation avec df_arrondissement

df = pd.concat(
    [
        df,
        df_arrondissements
    ],
    axis=0,
    ignore_index=False
)
df.shape

(1480926, 9)

In [31]:
df.head()

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
0,2012,Gaz,Résidentiel + Petits Professionnels,Résidentiel,803.0,54,47043,Buzet-sur-Baïse,47
1,2012,Gaz,Entreprises,Tertiaire,29935.0,5,47049,Casseneuil,47
2,2012,Gaz,Résidentiel + Petits Professionnels,Résidentiel,5075.0,231,47051,Castelculier,47
3,2012,Gaz,Entreprises,Industrie,15882.0,1,47054,Castelmoron-sur-Lot,47
4,2012,Gaz,Résidentiel + Petits Professionnels,Résidentiel,4264.0,137,47054,Castelmoron-sur-Lot,47


In [32]:
df.tail()

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
1475326,2019,Electricité,Entreprises,Tertiaire,977.376385,9,75000,Paris,75
1475327,2019,Electricité,Entreprises,Tertiaire,8009.467422,96,75000,Paris,75
1475328,2019,Electricité,Entreprises,Secteur Inconnu,238.648394,2,75000,Paris,75
1475329,2019,Electricité,Petits professionels,Tertiaire,76335.154839,9081,75000,Paris,75
1486068,2019,Gaz,Entreprises,Industrie,50420.000000,1,13000,Marseille,13


## Traitement des valeurs manquantes


In [33]:
df.isna().mean()

annee                             0.000000
filiere                           0.000000
libelle_categorie_consommation    0.026086
libelle_grand_secteur             0.000000
consommation                      0.000000
nombre_de_points                  0.000000
code_commune                      0.000000
libelle_commune                   0.000035
code_departement                  0.000000
dtype: float64

In [34]:
df['libelle_categorie_consommation'] = df['libelle_categorie_consommation'].fillna('Non renseigne')

In [35]:
df.isna().sum()

annee                              0
filiere                            0
libelle_categorie_consommation     0
libelle_grand_secteur              0
consommation                       0
nombre_de_points                   0
code_commune                       0
libelle_commune                   52
code_departement                   0
dtype: int64

In [37]:
df_missing_libelle_commune = df[df['libelle_commune'].isna()].sort_values(['code_commune'])
df_missing_libelle_commune

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
1383184,2017,Gaz,Entreprises,Industrie,6705.44,1,4417,NaN,4
607193,2015,Gaz,Résidentiel + Petits Professionnels,Résidentiel,0.00,0,49518,NaN,49
254347,2015,Gaz,Non renseigne,Secteur Inconnu,0.00,0,49518,NaN,49
793405,2016,Electricité,Entreprises,Industrie,0.00,0,67574,NaN,67
609183,2016,Electricité,Entreprises,Tertiaire,95.06,1,67574,NaN,67
559280,2014,Electricité,Non renseigne,Secteur Inconnu,0.00,0,67574,NaN,67
559278,2014,Electricité,Entreprises,Tertiaire,82.48,1,67574,NaN,67
559276,2014,Electricité,Entreprises,Industrie,0.00,0,67574,NaN,67
398063,2012,Electricité,Non renseigne,Secteur Inconnu,0.00,0,67574,NaN,67
398061,2012,Electricité,Entreprises,Industrie,0.00,0,67574,NaN,67


In [39]:
missing_code_commune = df_missing_libelle_commune['code_commune'].unique()

In [40]:
df[df['code_commune'].isin(missing_code_commune)][['code_commune', 'libelle_commune']].dropna().drop_duplicates().to_dict(orient='records')

[{'code_commune': 73257, 'libelle_commune': 'Les Belleville'}]

In [41]:
# on ne peut récupérer que cette ville

df.loc[df['code_commune'] == 73257, 'libelle_commune'] = 'Les Belleville'

In [42]:
df.isna().sum()

annee                              0
filiere                            0
libelle_categorie_consommation     0
libelle_grand_secteur              0
consommation                       0
nombre_de_points                   0
code_commune                       0
libelle_commune                   50
code_departement                   0
dtype: int64

In [43]:
# on peut supprimer le reste ? 
pd.DataFrame({
    'normal_dataset':df['consommation'].describe(),
    'missing_dataset': df_missing_libelle_commune['consommation'].describe()
}
)

,normal_dataset,missing_dataset
count,1.480926e+06,52.000000
mean,5.391949e+03,251.172308
std,6.384466e+04,1019.588497
min,-6.240000e+02,0.000000
25%,8.253000e+01,0.000000
50%,4.222731e+02,0.000000
75%,1.990297e+03,27.000000
max,1.529085e+07,6705.440000


In [44]:
# c'est négligeable on peut le supprimer
df = df[df.isna().sum(axis=1) == 0]

In [45]:
df.shape

(1480876, 9)

## Étude des variables consommation et nombre_de_points

In [41]:
df[['consommation', 'nombre_de_points']].describe(percentiles=np.arange(0, 1., .1))

,consommation,nombre_de_points
count,1.480876e+06,1.480876e+06
mean,5.392123e+03,2.978067e+02
std,6.384573e+04,2.636242e+03
min,-6.240000e+02,0.000000e+00
0%,-6.240000e+02,0.000000e+00
10%,6.619668e+00,1.000000e+00
20%,5.784940e+01,1.000000e+00
30%,1.157127e+02,1.000000e+00
40%,2.250091e+02,2.000000e+00
50%,4.223274e+02,4.000000e+00


In [46]:
# présence de valeurs négatives dans la consommation

df[df['consommation'] < 0]

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
196435,2017,Electricité,Non renseigne,Secteur Inconnu,-0.46,2,79196,Plaine-et-Vallées,79
490207,2016,Gaz,Résidentiel + Petits Professionnels,Résidentiel,-624.00,31,80652,Le Quesnel,80


In [48]:
# étude de Le Quesnel

df[df['code_commune'] == 80652].sort_values(['filiere', 'libelle_grand_secteur', 'annee'])

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
368237,2013,Electricité,Entreprises,Agriculture,359.0,5,80652,Le Quesnel,80
506736,2014,Electricité,Entreprises,Agriculture,327.0,5,80652,Le Quesnel,80
616439,2015,Electricité,Entreprises,Agriculture,517.0,6,80652,Le Quesnel,80
221005,2016,Electricité,Entreprises,Agriculture,465.0,6,80652,Le Quesnel,80
196161,2017,Electricité,Entreprises,Agriculture,604.0,6,80652,Le Quesnel,80
852654,2018,Electricité,Non renseigne,Agriculture,636.0,22,80652,Le Quesnel,80
1472844,2019,Electricité,Non renseigne,Agriculture,552.0,6,80652,Le Quesnel,80
594314,2013,Electricité,Entreprises,Industrie,0.0,0,80652,Le Quesnel,80
582838,2014,Electricité,Entreprises,Industrie,0.0,0,80652,Le Quesnel,80
700316,2015,Electricité,Entreprises,Industrie,0.0,0,80652,Le Quesnel,80


In [44]:
# on doit pouvoir changer le signe

In [49]:
# étude de Plaine Vallées

df[df['code_commune'] == 79196].sort_values(['filiere', 'libelle_grand_secteur', 'annee'])

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
265597,2015,Electricité,Entreprises,Agriculture,52.340,1,79196,Plaine-et-Vallées,79
721660,2016,Electricité,Entreprises,Agriculture,2009.250,20,79196,Plaine-et-Vallées,79
921777,2017,Electricité,Entreprises,Agriculture,2388.490,20,79196,Plaine-et-Vallées,79
929127,2017,Electricité,Entreprises,Agriculture,0.000,0,79196,Plaine-et-Vallées,79
852749,2018,Electricité,Non renseigne,Agriculture,0.000,0,79196,Plaine-et-Vallées,79
1067012,2018,Electricité,Non renseigne,Agriculture,2018.199,19,79196,Plaine-et-Vallées,79
1465118,2019,Electricité,Non renseigne,Agriculture,2144.163,20,79196,Plaine-et-Vallées,79
784133,2015,Electricité,Entreprises,Industrie,7339.610,145,79196,Plaine-et-Vallées,79
557361,2016,Electricité,Entreprises,Industrie,4421.180,125,79196,Plaine-et-Vallées,79
154600,2017,Electricité,Entreprises,Industrie,321.990,4,79196,Plaine-et-Vallées,79


In [46]:
# on peut le remettre en positif aussi
df.loc[:, 'consommation'] = df.loc[:, 'consommation'].abs()

In [47]:
df[['consommation', 'nombre_de_points']].describe(percentiles=np.arange(0, 1., .1))

,consommation,nombre_de_points
count,1.480876e+06,1.480876e+06
mean,5.392123e+03,2.978067e+02
std,6.384573e+04,2.636242e+03
min,0.000000e+00,0.000000e+00
0%,0.000000e+00,0.000000e+00
10%,6.619938e+00,1.000000e+00
20%,5.785000e+01,1.000000e+00
30%,1.157132e+02,1.000000e+00
40%,2.250094e+02,2.000000e+00
50%,4.223296e+02,4.000000e+00


In [48]:
df.sort_values('consommation').tail(50)

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
1158268,2016,Gaz,Entreprises,Industrie,4.777459e+06,4,44103,Montoir-de-Bretagne,44
977905,2014,Gaz,Entreprises,Industrie,4.790584e+06,4,76476,Port-Jérôme-sur-Seine,76
398062,2019,Gaz,Entreprises,Industrie,4.826398e+06,3,68020,Bantzenheim,68
977867,2014,Gaz,Entreprises,Industrie,4.886421e+06,4,68020,Bantzenheim,68
480762,2017,Gaz,Entreprises,Industrie,4.900199e+06,5,59183,Dunkerque,59
1094414,2018,Gaz,Entreprises,Industrie,4.999769e+06,3,59092,Bouchain,59
1205871,2013,Gaz,Entreprises,Industrie,5.010390e+06,4,68020,Bantzenheim,68
3207,2012,Gaz,Entreprises,Industrie,5.084090e+06,4,68020,Bantzenheim,68
1415075,2017,Gaz,Entreprises,Industrie,5.094245e+06,4,76476,Port-Jérôme-sur-Seine,76
953484,2015,Gaz,Entreprises,Industrie,5.157143e+06,4,68020,Bantzenheim,68


In [49]:
# https://fr.wikipedia.org/wiki/Liste_des_raffineries_fran%C3%A7aises
# Saint Avold semble contenir des usines de métallurgie
# Saint-Laurent de Cerdans semble surprenant... 

In [56]:
df_saint_laurent = pd.read_csv('conso_full.csv', sep=';')
df_saint_laurent = df_saint_laurent[df_saint_laurent['Libellé Commune'] == 'Saint-Laurent-de-Cerdans']
df_saint_laurent.shape

(33, 23)

In [57]:
df_saint_laurent[df_saint_laurent['Code catégorie consommation'] == 'RES-PRO'].sort_values(['Année', 'Libellé catégorie consommation'])

,Opérateur,Année,Filière,Code catégorie consommation,Libellé catégorie consommation,Code Grand secteur,Libellé Grand Secteur,Code NAF,Libellé Secteur NAF,Consommation (MWh),Nombre de points,Indice qualité,Thermosensibilité (MWh/degré-jour),Part Thermosensible,Nombre de mailles secretisées,Code Commune,Libellé Commune,Code EPCI,Libellé EPCI,Code Département,Libellé Département,Code Région,Libellé Région
421041,Régie électrique de Saint Laurent de Cerdans,2011,Electricité,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,4086787.00,729,1,0.0,0.0,0,66179,Saint-Laurent-de-Cerdans,246600548,CC du Haut Vallespir,66,Pyrénées-Orientales,76,Occitanie
20130,Régie électrique de Saint Laurent de Cerdans,2012,Electricité,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,3948016.00,734,1,0.0,0.0,0,66179,Saint-Laurent-de-Cerdans,246600548,CC du Haut Vallespir,66,Pyrénées-Orientales,76,Occitanie
367931,Régie électrique de Saint Laurent de Cerdans,2013,Electricité,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,3597225.00,739,1,0.0,0.0,0,66179,Saint-Laurent-de-Cerdans,246600548,CC du Haut Vallespir,66,Pyrénées-Orientales,76,Occitanie
362573,Régie électrique de Saint Laurent de Cerdans,2014,Electricité,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,6649416.00,917,1,0.0,0.0,0,66179,Saint-Laurent-de-Cerdans,246600548,CC du Haut Vallespir,66,Pyrénées-Orientales,76,Occitanie
784746,Régie électrique de Saint Laurent de Cerdans,2015,Electricité,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,2858.40,934,1,0.0,0.0,0,66179,Saint-Laurent-de-Cerdans,246600548,CC du Haut Vallespir,66,Pyrénées-Orientales,76,Occitanie
923648,Régie électrique de Saint Laurent de Cerdans,2017,Electricité,RES-PRO,Résidentiel + Petits Professionnels,R,Résidentiel,NaN,0,2686.28,916,1,0.0,0.0,0,66179,Saint-Laurent-de-Cerdans,246600548,CC du Haut Vallespir,66,Pyrénées-Orientales,76,Occitanie


In [58]:
# il semble y avoir une erreur dans les données: le différentiel 2014 -> 2015 n'a pas de sens
# on peut supprimer ces lignes

In [59]:
df = df[df['libelle_commune'] != 66179]

In [60]:
del df_saint_laurent

In [62]:
df.shape

(1480876, 9)

## Etude du nombre de points

In [63]:
df.sort_values('nombre_de_points').tail(60)

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
898033,2017,Electricité,Résidentiel + Petits Professionnels,Résidentiel,6.348258e+05,155115,59350,Lille,59
657306,2018,Electricité,Résidentiel,Résidentiel,5.659892e+05,160646,33063,Bordeaux,33
1125275,2019,Electricité,Résidentiel,Résidentiel,5.455486e+05,162348,33063,Bordeaux,33
883564,2018,Electricité,Résidentiel,Résidentiel,5.184096e+05,163000,34172,Montpellier,34
1466974,2019,Electricité,Résidentiel,Résidentiel,5.011290e+05,163380,67482,Strasbourg,67
409376,2013,Electricité,Résidentiel + Petits Professionnels,Résidentiel,7.088561e+05,165407,67482,Strasbourg,67
659560,2011,Electricité,Résidentiel + Petits Professionnels,Résidentiel,5.879200e+05,166113,75000,Paris,75
1352706,2019,Electricité,Résidentiel,Résidentiel,5.072316e+05,166337,34172,Montpellier,34
137241,2012,Electricité,Résidentiel + Petits Professionnels,Résidentiel,6.089695e+05,166349,75000,Paris,75
27638,2013,Electricité,Résidentiel + Petits Professionnels,Résidentiel,6.188197e+05,166622,75000,Paris,75


In [57]:
# chiffres assez surprenants au vue de la population de ces villes: Paris semblerait devoir être plus haut
# hypothèse: plus de grands appartements dans Paris ?
# on laisse comme ça ...

## Transformation sur les années

In [64]:
df_conso_moyenne = df.groupby(
    [col for col in df.columns if col not in ['annee', 'consommation', 'nombre_de_point']]
).mean().reset_index()

df_conso_moyenne = df_conso_moyenne.drop(['annee'], axis=1)

print(df_conso_moyenne.shape)
print(df.shape)
df_conso_moyenne.head()

(728166, 8)
(1480876, 9)


,filiere,libelle_categorie_consommation,libelle_grand_secteur,nombre_de_points,code_commune,libelle_commune,code_departement,consommation
0,Electricité,Entreprises,Agriculture,0,2003,Acy,2,0.0
1,Electricité,Entreprises,Agriculture,0,2012,Ambrief,2,0.0
2,Electricité,Entreprises,Agriculture,0,2020,Any-Martin-Rieux,2,0.0
3,Electricité,Entreprises,Agriculture,0,2022,Arcy-Sainte-Restitue,2,0.0
4,Electricité,Entreprises,Agriculture,0,2031,Aubenton,2,0.0


In [65]:
dfs = []

for col in ['filiere', 'libelle_categorie_consommation', 'libelle_grand_secteur']:
    for val in df_conso_moyenne[col].unique():
        df_temp = df_conso_moyenne[df_conso_moyenne[col]  == val].groupby(['code_commune', 'libelle_commune', 'code_departement']).sum().reset_index()
        
        df_temp = df_temp.rename(columns=
                                 {
                                     'consommation': 'consommation_{}_{}'.format(col, val).lower(),
                                     'nombre_de_points': 'nombre_de_points_{}_{}'.format(col, val).lower(),
                                 }
                                )
        
        dfs.append(df_temp)

In [66]:
dfs[0]

,code_commune,libelle_commune,code_departement,nombre_de_points_filiere_electricité,consommation_filiere_electricité
0,1001,L'Abergement-Clémenciat,1,2857,23448.248713
1,1002,L'Abergement-de-Varey,1,1659,9722.713779
2,1004,Ambérieu-en-Bugey,1,76099,663813.915433
3,1005,Ambérieux-en-Dombes,1,3292,24685.198000
4,1006,Ambléon,1,359,2011.133074
...,...,...,...,...,...
34466,95676,Villers-en-Arthies,95,2208,19195.374987
34467,95678,Villiers-Adam,95,2352,18745.738610
34468,95680,Villiers-le-Bel,95,90022,600373.046266
34469,95682,Villiers-le-Sec,95,739,6887.154396


In [69]:
# vérification
df_conso_moyenne[df_conso_moyenne['code_commune'] == 1001]

,filiere,libelle_categorie_consommation,libelle_grand_secteur,nombre_de_points,code_commune,libelle_commune,code_departement,consommation
20799,Electricité,Entreprises,Industrie,1,1001,L'Abergement-Clémenciat,1,52.530973
96535,Electricité,Entreprises,Tertiaire,2,1001,L'Abergement-Clémenciat,1,234.551723
209300,Electricité,Petits professionels,Agriculture,9,1001,L'Abergement-Clémenciat,1,271.251214
210684,Electricité,Petits professionels,Agriculture,10,1001,L'Abergement-Clémenciat,1,315.027040
224205,Electricité,Petits professionels,Industrie,0,1001,L'Abergement-Clémenciat,1,0.000000
252396,Electricité,Petits professionels,Secteur Inconnu,0,1001,L'Abergement-Clémenciat,1,0.000000
269554,Electricité,Petits professionels,Secteur Inconnu,3,1001,L'Abergement-Clémenciat,1,55.209301
316266,Electricité,Petits professionels,Tertiaire,17,1001,L'Abergement-Clémenciat,1,183.164882
383419,Electricité,Résidentiel,Résidentiel,345,1001,L'Abergement-Clémenciat,1,2495.540440
383467,Electricité,Résidentiel,Résidentiel,346,1001,L'Abergement-Clémenciat,1,2474.688139


In [68]:
df_conso_moyenne[df_conso_moyenne['code_commune'] == 1001]['consommation'].sum()

23448.2487130612

In [70]:
# concaténation des datasets

df2_conso_moyenne = df_conso_moyenne.groupby(['code_commune', 'libelle_commune', 'code_departement']).sum().reset_index()
df2_conso_moyenne = df2_conso_moyenne.rename(columns={'consommation': 'consommation_totale', 'nombre_de_points': 'nombre_de_points_total'})
df2_conso_moyenne.head()

,code_commune,libelle_commune,code_departement,nombre_de_points_total,consommation_totale
0,1001,L'Abergement-Clémenciat,1,2857,2.344825e+04
1,1002,L'Abergement-de-Varey,1,1659,9.722714e+03
2,1004,Ambérieu-en-Bugey,1,103418,1.269513e+06
3,1005,Ambérieux-en-Dombes,1,3292,2.468520e+04
4,1006,Ambléon,1,359,2.011133e+03


In [71]:
for df_temp in dfs:
    df2_conso_moyenne = pd.merge(
        left=df2_conso_moyenne,
        right=df_temp,
        on=['code_commune', 'libelle_commune', 'code_departement'],
        how='left'
    )
    
df2_conso_moyenne.head()

,code_commune,libelle_commune,code_departement,nombre_de_points_total,consommation_totale,nombre_de_points_filiere_electricité,consommation_filiere_electricité,nombre_de_points_filiere_gaz,consommation_filiere_gaz,nombre_de_points_libelle_categorie_consommation_entreprises,consommation_libelle_categorie_consommation_entreprises,nombre_de_points_libelle_categorie_consommation_non renseigne,consommation_libelle_categorie_consommation_non renseigne,nombre_de_points_libelle_categorie_consommation_petits professionels,consommation_libelle_categorie_consommation_petits professionels,nombre_de_points_libelle_categorie_consommation_résidentiel,consommation_libelle_categorie_consommation_résidentiel,nombre_de_points_libelle_categorie_consommation_résidentiel + petits professionnels,consommation_libelle_categorie_consommation_résidentiel + petits professionnels,nombre_de_points_libelle_grand_secteur_agriculture,consommation_libelle_grand_secteur_agriculture,nombre_de_points_libelle_grand_secteur_industrie,consommation_libelle_grand_secteur_industrie,nombre_de_points_libelle_grand_secteur_secteur inconnu,consommation_libelle_grand_secteur_secteur inconnu,nombre_de_points_libelle_grand_secteur_tertiaire,consommation_libelle_grand_secteur_tertiaire,nombre_de_points_libelle_grand_secteur_résidentiel,consommation_libelle_grand_secteur_résidentiel
0,1001,L'Abergement-Clémenciat,1,2857,2.344825e+04,2857,23448.248713,NaN,NaN,3.0,287.082696,NaN,NaN,39.0,824.652437,691.0,4970.228579,2124,17366.285000,19.0,586.278254,1.0,52.530973,3.0,55.209301,19.0,417.716605,2815,22336.513579
1,1002,L'Abergement-de-Varey,1,1659,9.722714e+03,1659,9722.713779,NaN,NaN,NaN,NaN,NaN,NaN,40.0,241.001449,328.0,1773.802330,1291,7707.910000,3.0,74.899964,NaN,NaN,0.0,0.000000,37.0,166.101485,1619,9481.712330
2,1004,Ambérieu-en-Bugey,1,103418,1.269513e+06,76099,663813.915433,27319.0,605698.721551,1150.0,357961.841629,5.0,1054.318333,2853.0,44932.941730,21649.0,160040.955292,77761,705522.580000,0.0,0.000000,335.0,38951.348807,468.0,4785.467174,3205.0,360212.285712,99410,865563.535292
3,1005,Ambérieux-en-Dombes,1,3292,2.468520e+04,3292,24685.198000,NaN,NaN,85.0,533.700000,191.0,2773.900000,NaN,NaN,1602.0,11001.688000,1414,10375.910000,76.0,540.749000,0.0,0.000000,144.0,2296.911000,56.0,469.940000,3016,21377.598000
4,1006,Ambléon,1,359,2.011133e+03,359,2011.133074,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,130.0,769.979740,229,1241.153333,0.0,0.000000,NaN,NaN,0.0,0.000000,0.0,0.000000,359,2011.133074


In [72]:
df2_conso_moyenne.notna().mean()

code_commune                                                                           1.000000
libelle_commune                                                                        1.000000
code_departement                                                                       1.000000
nombre_de_points_total                                                                 1.000000
consommation_totale                                                                    1.000000
nombre_de_points_filiere_electricité                                                   1.000000
consommation_filiere_electricité                                                       1.000000
nombre_de_points_filiere_gaz                                                           0.289026
consommation_filiere_gaz                                                               0.289026
nombre_de_points_libelle_categorie_consommation_entreprises                            0.786400
consommation_libelle_categorie_consommat

In [73]:
df2_conso_moyenne = df2_conso_moyenne.fillna(0)

In [74]:
df2_conso_moyenne.rename(columns={'code_commune': 'codgeo'}).\
to_csv('clean_data/conso_moyenne_clean.csv', index=False)

## Création de l'évolution

In [75]:
# comptage du nombre de lignes par commune
df[['code_commune', 'annee']].groupby([ 'code_commune']).nunique().reset_index().head(10)

,code_commune,annee
0,1001,9
1,1002,9
2,1004,9
3,1005,4
4,1006,9
5,1007,9
6,1008,9
7,1009,9
8,1010,9
9,1011,9


In [76]:
# on peut voir que certaines communes n'ont que 4 années renseignées
# tentative pour voir quelles sont les années les plus représentées

df[['annee', 'code_commune']].groupby(['annee']).nunique().reset_index().head(10)

,annee,code_commune
0,2011,32706
1,2012,32867
2,2013,33247
3,2014,33250
4,2015,34309
5,2016,34381
6,2017,34461
7,2018,34433
8,2019,34441


In [77]:
# essayons de comparer les années 2018 et 2019

df_2018 = df[df['annee'].isin([2018])]
df_2019 = df[df['annee'].isin([2019])]

communes_2018 = set(df_2018['code_commune'].unique())
communes_2019 = set(df_2019['code_commune'].unique())

# communes présentes en 2018 mais pas en 2019
differences_2018_2019 = communes_2018.difference(communes_2019)
differences_2018_2019

{40001, 73047}

In [78]:
# communes présentes en 2019 mais pas en 2018
differences_2019_2018 = communes_2019.difference(communes_2018)
differences_2019_2018

{55138, 67006, 67021, 74042, 74256, 88291, 91100, 91412, 91654, 91671}

In [79]:
df[df['code_commune'].isin(differences_2018_2019)]

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
20114,2012,Electricité,Entreprises,Industrie,4977.0,5,40001,Aire-sur-l'Adour,40
20115,2012,Electricité,Entreprises,Tertiaire,16803.0,607,40001,Aire-sur-l'Adour,40
20116,2012,Electricité,Résidentiel + Petits Professionnels,Résidentiel,18291.0,3364,40001,Aire-sur-l'Adour,40
78879,2013,Gaz,Entreprises,Tertiaire,37800.0,168,40001,Aire-sur-l'Adour,40
78880,2013,Gaz,Résidentiel + Petits Professionnels,Résidentiel,30485.0,1804,40001,Aire-sur-l'Adour,40
...,...,...,...,...,...,...,...,...,...
1066674,2018,Electricité,Non renseigne,Tertiaire,4988.0,524,40001,Aire-sur-l'Adour,40
1066675,2018,Electricité,Non renseigne,Secteur Inconnu,1328.0,143,40001,Aire-sur-l'Adour,40
1204345,2018,Electricité,Non renseigne,Agriculture,1234.0,285,40001,Aire-sur-l'Adour,40
1440059,2018,Gaz,Non renseigne,Tertiaire,15114.0,99,40001,Aire-sur-l'Adour,40


In [80]:
df[df['code_commune'].isin(differences_2019_2018)]

,annee,filiere,libelle_categorie_consommation,libelle_grand_secteur,consommation,nombre_de_points,code_commune,libelle_commune,code_departement
4065,2013,Electricité,Entreprises,Tertiaire,0.0,0,67006,Altenheim,67
145716,2015,Electricité,Entreprises,Industrie,0.0,0,91100,Bouville,91
145717,2015,Electricité,Entreprises,Tertiaire,1417.0,1,91100,Bouville,91
145718,2015,Electricité,Résidentiel + Petits Professionnels,Résidentiel,2420.0,325,91100,Bouville,91
145734,2015,Electricité,Entreprises,Industrie,0.0,0,91412,Mondeville,91
...,...,...,...,...,...,...,...,...,...
1467202,2019,Electricité,Non renseigne,Tertiaire,0.0,0,91412,Mondeville,91
1467209,2019,Electricité,Non renseigne,Agriculture,613.0,3,91654,Videlles,91
1467210,2019,Electricité,Résidentiel,Résidentiel,3265.0,305,91671,Villeneuve-sur-Auvers,91
1467236,2019,Gaz,Résidentiel,Résidentiel,43547.0,2244,67021,Barr,67


In [74]:
# trop compliqué pour l'instant